In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, fftshift, ifft2

In [2]:
# Input image path
input_image_path = 'F:/B208/AUTO-count_CNT/DeepLearning assists CNT orientation counting/Figure1/label.png'
# Output image paths
output_image_path = 'F:/B208/AUTO-count_CNT/DeepLearning assists CNT orientation counting/Figure1/fft_magnitude_angle_relationship.png'
output_histogram_path = 'F:/B208/AUTO-count_CNT/DeepLearning assists CNT orientation counting/Figure1/fft_angle_histogram.png'
output_txt_path = 'F:/B208/AUTO-count_CNT/DeepLearning assists CNT orientation counting/Figure1/fft_angle_distribution.txt'  # Path for the output TXT file
output_inverse_image_path = 'F:/B208/AUTO-count_CNT/DeepLearning assists CNT orientation counting/Figure1/inverse_fft_image.png'  # Path for the inverse FFT output image

In [3]:
# Read the image
image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)

# If the image fails to read, raise an error
if image is None:
    raise FileNotFoundError(f"Image file not found at {input_image_path}")

# Perform Fourier Transform
f_transform = fft2(image)
f_transform_shifted = fftshift(f_transform)

# Calculate Magnitude Spectrum
magnitude_spectrum = np.abs(f_transform_shifted)
log_magnitude_spectrum = np.log(magnitude_spectrum + 1)

# Calculate Phase Spectrum
phase_spectrum = np.angle(f_transform_shifted)

# Frequency component coordinates
height, width = magnitude_spectrum.shape
center = (height // 2, width // 2)
y, x = np.indices((height, width))
freq_x = x - center[1]
freq_y = y - center[0]

# Filter out small magnitude values
threshold = 0.20 * magnitude_spectrum.max()
magnitude_spectrum[magnitude_spectrum < threshold] = 0

# Find peak positions
peaks = np.where(magnitude_spectrum > 0)
angles = []
for px, py in zip(peaks[0], peaks[1]):
    freq_x_val = freq_x[px, py]
    freq_y_val = freq_y[px, py]
    angle = np.degrees(np.arctan2(freq_y_val, freq_x_val))
    angle = (angle + 180) % 180  # Convert to 0-180 range
    angle = (angle + 90) % 180   # Add 90 degrees
    angles.append(round(angle))  # Round to the nearest integer

FileNotFoundError: Image file not found at F:/B208/AUTO-count_CNT/DeepLearning assists CNT orientation counting/Figure1/label.png

In [ ]:
# 绘制幅度谱
plt.figure(figsize=(6, 6))  # 创建一个新图形
plt.imshow(log_magnitude_spectrum, cmap='gray')
plt.axis('off')  # 关闭坐标轴
plt.show()  # 显示幅度谱图

# 绘制相位谱
plt.figure(figsize=(6, 6))  # 创建一个新图形
plt.imshow(phase_spectrum, cmap='hsv')
plt.axis('off')  # 关闭坐标轴
plt.show()  # 显示相位谱图

# Save Magnitude and Phase Spectrum Images
plt.tight_layout()
plt.savefig(output_image_path)
plt.show()

# Prepare for histogram
angle_bins = np.linspace(0, 180, 181)  # 180 bins for angles
pixel_distribution, _ = np.histogram(angles, bins=angle_bins)

# Normalize the distribution
normalized_distribution = pixel_distribution / np.sum(pixel_distribution)

# Calculate bin centers
bin_centers = (angle_bins[:-1] + angle_bins[1:]) / 2

# Calculate ratio for 88-92 degrees
within_range_mask = (bin_centers >= 88) & (bin_centers <= 92)
ratio_88_92 = np.sum(normalized_distribution[within_range_mask])

# Save angle distribution to TXT file
with open(output_txt_path, 'w') as f:
    f.write('Angle (degrees), Pixel Count\n')
    for angle, count in zip(bin_centers, pixel_distribution):
        f.write(f'{angle:.2f}, {count}\n')

: 

In [ ]:
# Plot normalized angle histogram
plt.figure(figsize=(6, 6), dpi=100)
plt.bar(bin_centers, normalized_distribution, width=np.diff(angle_bins), edgecolor='black', color='red', align='center')
plt.xlabel('Angle (degrees)', fontfamily='arial')
plt.ylabel('Percentage of Pixels', fontfamily='arial')
plt.title('Theoretical Pixel Distribution by Angle', fontfamily='arial')

# Add shaded area for 88-92 degrees
plt.axvspan(88, 92, color='red', alpha=0.3, label='88-92 degrees')

# Annotate the 88-92 degree ratio
plt.text(10, 0.9, f'88-92° Ratio: {ratio_88_92:.4f}', color='black', fontsize=12, fontfamily='arial')

plt.ylim(0, 1.0)
plt.xlim(0, 180)
plt.legend()
plt.tight_layout()
plt.savefig(output_histogram_path)  # Save the normalized angle histogram image
plt.show()

# Inverse Fourier Transform to obtain the denoised image
# Set small values in the magnitude spectrum to zero to reduce noise
denoised_magnitude_spectrum = magnitude_spectrum.copy()
denoised_magnitude_spectrum[magnitude_spectrum < threshold] = 0

# Reconstruct the complex spectrum using the filtered magnitude and original phase
filtered_spectrum = denoised_magnitude_spectrum * np.exp(1j * phase_spectrum)
inverse_image = np.abs(ifft2(filtered_spectrum))

# Normalize and convert to uint8 for displaying/saving
inverse_image_normalized = cv2.normalize(inverse_image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# Save the inverse FFT image
cv2.imwrite(output_inverse_image_path, inverse_image_normalized)

# Display the inverse FFT image
plt.figure(figsize=(6, 6))
plt.imshow(inverse_image_normalized, cmap='gray')
plt.axis('off')  # Turn off the axis
plt.title('Inverse FFT Image')
plt.show()

print(f"Output image saved at {output_image_path}")
print(f"Histogram image saved at {output_histogram_path}")
print(f"Angle distribution saved at {output_txt_path}")
print(f"Inverse FFT image saved at {output_inverse_image_path}")

: 